# Affiliation Parsing

Our goal is to parse the affiliation text into country and insititution. For example, "McCombs School of Business, The University of Texas at Austin Austin, TX 78712 U.S.A." should be parsed to "University of Texas at Austin" and "USA". We don't care too much about the department and address for now. 

All rule based methods.

## Country Parsing

- use name search via ISO country list + add additional country names, such as usa, uk, korea, etc.
- handle special entities such as Georgia as a state and a country), etc: https://en.wikipedia.org/wiki/List_of_homonymous_states_and_regions
- handle no country 
    - many affiliation with no country is actually usa
    - special cases

## Institution Parsing

- split the string by comma and use keywords like "university", "institute" to search for organization 


## Data

The input of this is the latest database and the latest processed affiliation csv. The program will generate a new csv only for the not processed data. Given the total database is only around 300m, this approach should work for the forseeable future. 

In [1]:
import sqlite3
import numpy as np
import pandas as pd
# pd.set_option('max_colwidth', -1)  # set max pd column with to unlimited

In [2]:
base_file_name = '05-30-2020'
database_file_name = '../data/pubcrawler-' + base_file_name + '.db'
aff_file_name = '../data/affiliation-' + base_file_name + '.csv'

print(database_file_name, aff_file_name)

../data/pubcrawler-05-30-2020.db ../data/affiliation-05-30-2020.csv


In [3]:
# use the lastest database
#conn = sqlite3.connect("../data/pubcrawler-05-30-2020.db")

# use the sample databse
conn = sqlite3.connect("../data/pub_raw.db")

In [4]:
# sample 500 rows for analysis
authors = pd.read_sql_query("select * from author order by random() limit 500;", conn)

# the following retrieve all rows
#authors = pd.read_sql_query("select * from author;", conn)
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   full_name    500 non-null    object
 2   first_name   500 non-null    object
 3   last_name    500 non-null    object
 4   middle_name  204 non-null    object
 5   email        500 non-null    object
 6   affiliation  500 non-null    object
dtypes: int64(1), object(6)
memory usage: 27.5+ KB


In [5]:
authors.head()

,id,full_name,first_name,last_name,middle_name,email,affiliation
0,1736,"Zigurs, Ilze",Ilze,Zigurs,None,nan,nan
1,2392,"Ginzberg, Michael J.",Michael,Ginzberg,J.,nan,Weatherhead School of Management Case Western ...
2,1961,"George, Joey F.",Joey,George,F.,jgeorge@cob.fsu.edu,Department of Information & Management Science...
3,1080,"Ellen Moore, Jo",Jo,Ellen Moore,None,joemoor@siue.edu,"Southern Illinois University Edwardsville, Cam..."
4,244,"Lynne Markus, M.",M.,Lynne Markus,None,MLMarkus@bentley.edu,"Information and Process Management Department,..."


## Prepare Country and Insititution Lists

- ISO country list is composed from pycountry package
- Grid institution list is from https://www.grid.ac/ - processed using the `process-grid.ipynb`

In [6]:
# compose the country name list from pycountry
#pd.set_option('max_colwidth', 800) # enlarge the cell width if needed
import pycountry

country_list = []
for c in pycountry.countries:
    country_list.append(c.name.lower()) # change to all lowercase

# add additional country spellings
# all lower case

additional_country_list = ['usa', 'korea'] # all lowercase
country_list.extend(additional_country_list)

print(country_list)

['aruba', 'afghanistan', 'angola', 'anguilla', 'åland islands', 'albania', 'andorra', 'united arab emirates', 'argentina', 'armenia', 'american samoa', 'antarctica', 'french southern territories', 'antigua and barbuda', 'australia', 'austria', 'azerbaijan', 'burundi', 'belgium', 'benin', 'bonaire, sint eustatius and saba', 'burkina faso', 'bangladesh', 'bulgaria', 'bahrain', 'bahamas', 'bosnia and herzegovina', 'saint barthélemy', 'belarus', 'belize', 'bermuda', 'bolivia, plurinational state of', 'brazil', 'barbados', 'brunei darussalam', 'bhutan', 'bouvet island', 'botswana', 'central african republic', 'canada', 'cocos (keeling) islands', 'switzerland', 'chile', 'china', "côte d'ivoire", 'cameroon', 'congo, the democratic republic of the', 'congo', 'cook islands', 'colombia', 'comoros', 'cabo verde', 'costa rica', 'cuba', 'curaçao', 'christmas island', 'cayman islands', 'cyprus', 'czechia', 'germany', 'djibouti', 'dominica', 'denmark', 'dominican republic', 'algeria', 'ecuador', 'egy

In [7]:
# clean up the text
def clean_text(aff_text):
    aff_text = aff_text.lower() # change to lower case 
    aff_text = aff_text.replace('.', '')  # remove all dots
    return aff_text

In [8]:
# find country by matching the country name from a list
def find_country(aff_text):
    # Test if a string (aff_text) contains element e from a list (country_list)
    result = [e for e in country_list if(e in aff_text)] 
    
    if len(result) > 0: # found a country match
        return result # return country list
    else: 
        return 'no country found'

In [9]:
# find country testing example
country_example = 'Department of Design & Innovation, The Weatherhead School of Management, Case Western Reserve University, 10900 Euclid Avenue, Cleveland, Ohio 44106-7235 U.S.A. | Warwick Business School, University of Warwick, Coventry CV4 7RL UNITED KINGDOM'

aff_text = clean_text(country_example)
country = find_country(aff_text)

if country == 'no country found':
    print('no luck')
else:
    print(country)

['united kingdom', 'usa']


In [10]:
# grid
grid = pd.read_csv('../data/grid-parsed.csv')
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97795 entries, 0 to 97794
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           97795 non-null  object
 1   wikipedia_url  33274 non-null  object
 2   links          97795 non-null  object
 3   city           97795 non-null  object
 4   state          36035 non-null  object
 5   country        97795 non-null  object
dtypes: object(6)
memory usage: 4.5+ MB


In [11]:
# change all to lowercase
grid['name'] = grid['name'].str.lower()
grid['city'] = grid['city'].str.lower()
grid['state'] = grid['state'].str.lower()
grid['country'] = grid['country'].str.lower()

In [12]:
# get name list
grid_name_list = grid['name'].tolist()
grid_name_list[0:5]

['australian national university',
 'monash university',
 'university of queensland',
 'macquarie university',
 'unsw sydney']

In [13]:
grid.head()

,name,wikipedia_url,links,city,state,country
0,australian national university,http://en.wikipedia.org/wiki/Australian_Nation...,['http://www.anu.edu.au/'],canberra,australian capital territory,australia
1,monash university,http://en.wikipedia.org/wiki/Monash_University,['http://www.monash.edu/'],melbourne,victoria,australia
2,university of queensland,http://en.wikipedia.org/wiki/University_of_Que...,['http://www.uq.edu.au/'],brisbane,queensland,australia
3,macquarie university,http://en.wikipedia.org/wiki/Macquarie_University,['http://mq.edu.au/'],sydney,new south wales,australia
4,unsw sydney,http://en.wikipedia.org/wiki/University_of_New...,['https://www.unsw.edu.au/'],sydney,new south wales,australia


In [14]:
# given a org name, get the country
grid[grid['name']=='australian national university']['country'].tolist()[0]

'australia'

In [15]:
# find org by splitting the string and do a keyword based matching
def find_org(aff_text):
    aff_text = aff_text.replace('|', ',') # replace the pipe symbol with comma for splitting
    phrases = aff_text.split(',') # split by comma
    phrases = [p.strip().lower() for p in phrases] # strip and change to lower case

    # org keywords
    org_keywords = ['university', 'institute']

    # p is a phrase after splitting such as "case western reserve university", k is a keyword in the list above
    result = [p for p in phrases if any(k in p for k in org_keywords)]
    result = list(set(result))  # remove duplicates if any

    if len(result) > 0: # found a org match
        return result # return org list
    else:
        return 'no org found'

In [16]:
# find org by splitting the string and do a keyword based matching
def find_org_grid(aff_text):
    aff_text = aff_text.replace('|', ',') # replace the pipe symbol with comma for splitting
    phrases = aff_text.split(',') # split by comma
    phrases = [p.strip().lower() for p in phrases] # strip and change to lower case

    # org keywords
    #org_keywords = ['university', 'institute']
    org_keywords = grid_name_list
    # p is a phrase after splitting such as "case western reserve university", k is a keyword in the list above
    result = [p for p in phrases if any(k == p for k in org_keywords)]
    #result = [k for k in org_keywords if any(k in p for p in phrases)]
    result = list(set(result))  # remove duplicates if any

    if len(result) > 0: # found a org match
        return result # return org list
    else:
        return 'no grid org found'

In [17]:
# find institution example
institute_example = 'Department of Design & Innovation, The Weatherhead School of Management, Case Western Reserve University, 10900 Euclid Avenue, Cleveland, Ohio 44106-7235 U.S.A. | Warwick Business School, University of Warwick, Coventry CV4 7RL UNITED KINGDOM'

org = find_org(institute_example)
org_grid = find_org_grid(institute_example)
print(org, org_grid)

['case western reserve university', 'university of warwick'] ['case western reserve university', 'university of warwick']


In [18]:
%%time
# About wall time: 2min 30s for full dataset
# parsing the country and organization 
current_aff = {}
all_aff =[] # final country info dict
country_match = 0
org_match = 0
for i in range(len(authors)): # loop over all authors
    aff_text = authors['affiliation'][i]  # get author affiliation string
    
    # clean up the text
    aff_text = clean_text(aff_text) 

    # find country
    country = find_country(aff_text) 
    print('returned country: ', country)
    
    if country == 'no country found':
        current_aff = {
            'country': '', # country null
            'multi_country': '', # multiple country null
            }
    else:
        country_match += 1
        if len(country) > 1:  # multiple country found
            # print('multiple country found:', country)

            current_aff = {
                'country': '', 
                'multi_country': country,
                }
        else: # a single country is found
            #print(country[0]) 
            current_aff = {
                'country': country[0], 
                'multi_country': '',
                }

    # find org 
    
    org = find_org_grid(aff_text)
    print('returned org: ', org)
    
    if org == 'no grid org found': # search based on grid dataset found no match
        org = find_org(aff_text) # search based on keywords
        if org == 'no org found':
            current_aff.update(org='', multi_org='') # no org found
        else:
            org_match += 1
            if len(org) > 1:  # multiple org found
                print('multiple org found:', org)
                current_aff.update(org='', multi_org=org)
            else:
                #print(org[0]) # a single org is found
                current_aff.update(org=org[0], multi_org='')
    else:
        org_match += 1
        if len(org) > 1:  # multiple org found
            print('multiple org found:', org)
            current_aff.update(org='', multi_org=org)
        else:
            #print(org[0]) # a single org is found
            # update the country using grid country if not country found
            country_grid = grid[grid['name']==org[0]]['country'].tolist()[0]
            if country == 'no country found': # set country and set multi-country to none
                current_aff.update(org=org[0], multi_org='', country=country_grid, multi_country='')
            else: 
                current_aff.update(org=org[0], multi_org='')
    all_aff.append(current_aff)

country:  no country found
returned org:  no grid org found
returned country:  no country found
returned org:  ['university of minnesota']
returned country:  ['germany']
returned org:  no grid org found
multiple org found: ['institute for marketing and media', 'university of hamburg']
returned country:  ['usa']
returned org:  no grid org found
returned country:  no country found
returned org:  no grid org found
returned country:  ['usa']
returned org:  no grid org found
returned country:  ['netherlands']
returned org:  ['erasmus university rotterdam']
returned country:  no country found
returned org:  no grid org found
returned country:  no country found
returned org:  no grid org found
returned country:  no country found
returned org:  ['college of business administration', 'oklahoma state university']
multiple org found: ['college of business administration', 'oklahoma state university']
returned country:  no country found
returned org:  no grid org found
returned country:  ['singapo

In [19]:
# sample data results
# country match 2611, org match (grid) 3382 about 2 minutes
# country match 2611, org match (keyword) 4611 about 5 sec.
print(country_match, org_match)

197 376


In [20]:
# convert the resulting dict to pandas dataframe
import pandas as pd

aff_parsed = pd.DataFrame(all_aff)
aff_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country        500 non-null    object
 1   multi_country  500 non-null    object
 2   org            500 non-null    object
 3   multi_org      500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [21]:
# add new columns to the exsting df
authors_parsed = pd.concat([authors, aff_parsed], axis=1)

## Georgia and USA 

handle georgia as a state and a country

- multi country has georgia and other country - remove georgia and use other country as the country

- match org with grid and use the found country

In [22]:
# make a test set
georgia_test = authors_parsed[authors_parsed['org'].str.contains('georgia')].copy()  
georgia_test.head()

,id,full_name,first_name,last_name,middle_name,email,affiliation,country,multi_country,org,multi_org
23,3280,"Slaughter, Sandra A.",Sandra,Slaughter,A.,asang@ntu.edu.sg,"Scheller College of Business, Georgia Institut...",georgia,,georgia institute of technology,
33,3808,"Storey, Veda C.",Veda,Storey,C.,vstorey@cis.gsu.edu,"Computer Information Systems Department, J. Ma...",georgia,,georgia state university,
124,1448,"Gallivan, Michael J.",Michael,Gallivan,J.,mgallivan@gsu.edu,"Computer Information Systems Department, J. Ma...",,"[georgia, usa]",georgia state university,
128,4034,"Thomas, Helen",Helen,Thomas,None,helen@loochi.mgt.gatech.edu,"DuPree College of Management, Georgia Institut...",georgia,,georgia institute of technology,
138,4922,"Sarkar, Sumantra",Sumantra,Sarkar,None,nan,"J. Mack Robinson College of Business, Georgia ...",georgia,,georgia state university,


In [23]:
# georgia rules
for index, row in georgia_test.iterrows():
    # georgia rule 1: if georgia is in multi_country - use the other coutnry as the country
    multi_country = row['multi_country']

    if 'georgia' in multi_country:
        multi_country.remove('georgia')
        georgia_test.at[index, 'country'] = multi_country[0]
        georgia_test.at[index, 'multi_country'] = ''
    
    # rule 2: match org and use grid country as country
    current_org = row['org']
    grid_matched_country = grid[grid['name']==current_org]['country']
    if len(grid_matched_country) > 0:  # found a match
        georgia_test.at[index, 'country'] = grid_matched_country.values[0]

In [24]:
# if email ends with .edu, change country to united states
edu_email_test = authors_parsed[authors_parsed['email'].str.endswith("edu")]

for index, row in edu_email_test.iterrows():
        edu_email_test.at[index, 'country'] = 'united states'

edu_email_test.head()

,id,full_name,first_name,last_name,middle_name,email,affiliation,country,multi_country,org,multi_org
2,1961,"George, Joey F.",Joey,George,F.,jgeorge@cob.fsu.edu,Department of Information & Management Science...,united states,,florida state university,
3,1080,"Ellen Moore, Jo",Jo,Ellen Moore,None,joemoor@siue.edu,"Southern Illinois University Edwardsville, Cam...",united states,,southern illinois university edwardsville,
4,244,"Lynne Markus, M.",M.,Lynne Markus,None,MLMarkus@bentley.edu,"Information and Process Management Department,...",united states,,bentley university,
8,4053,"Nault, Barrie R.",Barrie,Nault,R.,nault@cob.ohio-state.edu,"Fisher College of Business, Ohio State Univers...",united states,,ohio state university,
15,623,"Leonardi, Paul M.",Paul,Leonardi,M.,Leonardi@tmp.ucsb.edu,nan,united states,,,


In [25]:
# if country is empty, default to united states based on sample data analysis
for index, row in edu_email_test.iterrows():
        if row['country'] == '':  # country empty
            edu_email_test.at[index, 'country'] = 'united states'

In [26]:
# no org identified
no_org_df = authors_parsed[(authors_parsed['org']=='') & (authors_parsed['multi_org']=='')]
# no country identified
no_country_df = authors_parsed[(authors_parsed['country']=='') & (authors_parsed['multi_country']=='')]

In [27]:
print(len(no_country_df), len(no_org_df))

164 124


In [52]:
from email2country import email2country, email2institution_country
email = 'test@tju.edu.cn'
email_country = email2institution_country(email)
if email_country:
    print(email_country.lower())
else:
    print('no found')

china


In [53]:
# post processing handles the followings:
# - georgia and country
# - email to country rule
# - empty country rule
# - acronym rule (nyu, mit, etc.)
# - institution names that are not seperated by comma

from email2country import email2institution_country

def postprocessing(df):
    for index, row in df.iterrows():
        # empty country rule 1: if country is empty, default to united states based on sample data analysis
        if row['country'] == '':  # country empty
            df.at[index, 'country'] = 'united states'
        
        # georgia rule 1: if georgia is in multi_country - use the other coutnry as the country
        multi_country = row['multi_country']

        if 'georgia' in multi_country:
            multi_country.remove('georgia')
            df.at[index, 'country'] = multi_country[0]
            df.at[index, 'multi_country'] = ''
        
        # georgia rule 2: match org and use grid country as country
        current_org = row['org']
        grid_matched_country = grid[grid['name']==current_org]['country']
        if len(grid_matched_country) > 0:  # found a match
            df.at[index, 'country'] = grid_matched_country.values[0]

        # email rules
        if row['email'] != 'nan':
            country = email2institution_country(row['email'])
            if country:
                df.at[index, 'country'] = country.lower()
        
        # use grid list to match the affiliation (previously was seperated by comma)
        # "Center for Information Management Studies Babson College" - we want to match "babson college"
        if row['org']=='':
            short_org_names = ['nyu', 'mit', 'cmu', 'ucla', 'ibm']
            aff = row['affiliation'].lower()
            # p is a phrase after splitting such as "case western reserve university", k is a keyword in the list above
            matched_org = [p for p in grid_name_list if p in aff]
            if len(matched_org) > 0: # found a org match
                df.at[index, 'org'] = matched_org[0]

In [54]:
postprocessing(authors_parsed)

[Info] Country not found for "cob.ohio-state.edu"
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Email domain "hotmail.com" is generic. There is no specific country.
[Info] Country not found for "tntec.edu"
[Info] Email domain "hotmail.com" is generic. There is no specific country.
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Country not found for "hbs.edu"
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Email domain "gmail.com" is generic. There is no specific country.
[Info] Country not found for "ny.frb.org"
[Info] Email domain "gmail.com" is generic. There is no specific country.


In [55]:
# no org identified
no_org_df = authors_parsed[(authors_parsed['org']=='') & (authors_parsed['multi_org']=='')]
# no country identified
no_country_df = authors_parsed[(authors_parsed['country']=='') & (authors_parsed['multi_country']=='')]
print(len(no_country_df), len(no_org_df))

0 112


In [56]:
no_org_df.head(50)

,id,full_name,first_name,last_name,middle_name,email,affiliation,country,multi_country,org,multi_org
0,1736,"Zigurs, Ilze",Ilze,Zigurs,None,nan,nan,united states,,,
10,4119,"Dedrick, Jason",Jason,Dedrick,None,nan,nan,united states,,,
15,623,"Leonardi, Paul M.",Paul,Leonardi,M.,Leonardi@tmp.ucsb.edu,nan,united states,,,
24,3247,"Murray, Alan",Alan,Murray,None,alan@coriolisventures.com,"Coriolis Labs, New York, New York 10021",united states,,,
25,4798,"Deng, Jing",Jing,Deng,None,nan,nan,united states,,,
27,363,"Durcikova, Alexandra",Alexandra,Durcikova,None,alex@ou.edu,nan,united states,,,
32,512,"Greenwood, Brad N.",Brad,Greenwood,N.,greenwood@temple.edu,nan,united states,,,
34,460,"Cheng, Hsing Kenneth",Hsing,Cheng,Kenneth,kenny.cheng@warrington.ufl.edu,nan,united states,,,
41,4515,"Dong, Wei",Wei,Dong,None,weidong1@mail.ustc.edu.cn,nan,china,,,
45,4414,"Qahri-Saremi, Hamed",Hamed,Qahri-Saremi,None,hamed.saremi@depaul.edu,nan,united states,,,


In [57]:
# save parsed authors to csv
authors_parsed.to_csv('../data/authors-parsed.csv', index=False)